# Processing/Regridding CESM inputs and outputs

---

## Regrid between rectilinear grids with xesmf

Regridding SE and FV output and emissions

Regridding CESM ocean T grids / U grids

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import cartopy.crs as ccrs
import numpy as np

### Input data

In [ ]:
data_vars = ['SST']
ds1 = xr.open_mfdataset("/glade/scratch/emmons/PYTHON_TUTORIAL/f.e21.FWHISTBgcCrop.f09_f09_mg17.CMIP6-AMIP-WACCM.001.cam.h0.2013-*.nc")
drop_vars = set(ds1.data_vars) - set(data_vars)
dset = ds1.drop(drop_vars).load()
dset

In [ ]:
dr = dset['SST'] # get a DataArray

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
dr.isel(time=3).plot.pcolormesh(ax=ax, vmin=250, vmax=370, cmap='jet')
ax.coastlines()

### Input grid

In [ ]:
dset['lat'].data.shape, dset['lon'].data.shape

### Output grid

Let's downsample our data to 0.5∘×0.5∘. Just define the output grid as an xarray 

In [ ]:
dset_out = xr.Dataset({'lat': (['lat'], np.arange(-90, 90, 0.5)),
                     'lon': (['lon'], np.arange(0, 360, 0.5)),
                    }
                   )
dset_out

### Perform regridding

Make a regridder by `xe.Regridder(grid_in, grid_out, method)`. `grid` is just an xarray DataSet containing lat and lon values.

In [ ]:
import xesmf as xe

In [ ]:
regridder = xe.Regridder(dset, dset_out, method='bilinear')
regridder

The regridder says it can transform data from shape (192, 288)  to shape (360, 720).

Once the regridder object is created, regridding our input `DataArray` is straightforward:

In [ ]:
dr_out = regridder(dr)
dr_out

### Check results

The regridding result is consistent with the original data, with a much finer resolution:



In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
dr_out.isel(time=3).plot.pcolormesh(ax=ax, vmin=250, vmax=370, cmap='jet');
ax.coastlines();

### Broadcasting over extra dimensions
    
xESMF tracks coordinate values over extra dimensions, since horizontal regridding should not affect them.

In [ ]:
dr_out['time']

In [ ]:
# exactly the same as input
xr.testing.assert_identical(dr_out['time'], dset['time'])

We can plot the time series at a specific location, to make sure the broadcasting is correct:

In [ ]:
dr.sel(lon=260, lat=40, method='nearest').plot()  # input data

In [ ]:
dr_out.sel(lon=260, lat=40, method='nearest').plot()

Creating timeseries from averages over regions